In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import pickle

# --- Step 1: Load dataset ---
df = pd.read_csv("/content/fabric_recommendation_dataset.csv")

# --- Step 2: Features and target ---
X = df[["WorkLevel", "Weather", "SkinTone", "Season"]]
y = df["Fabric"]

# --- Step 3: Create pipeline ---
categorical_features = ["WorkLevel", "Weather", "SkinTone", "Season"]
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)]
)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(
        n_estimators=500,
        random_state=42
    ))
])

# --- Step 4: Split for training/testing ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Step 5: Train pipeline ---
pipeline.fit(X_train, y_train)

# --- Step 6: Top-3 Accuracy calculation ---
probs_test = pipeline.predict_proba(X_test)
classes = pipeline.classes_

top3_correct = sum(
    y_test.iloc[i] in classes[np.argsort(probs_test[i])[-3:][::-1]]
    for i in range(len(y_test))
)
top3_accuracy = top3_correct / len(y_test)
print(f"\n Accuracy: {top3_accuracy*100:.2f}%")

# --- Step 7: Save pipeline ---
with open("fabric_ml_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)
print("✅ Model saved as 'fabric_ml_pipeline.pkl'")

# --- Step 8: Recommend top fabric ---
def recommend_fabric(work, weather, skin, season):
    with open("fabric_ml_pipeline.pkl", "rb") as f:
        model = pickle.load(f)

    input_df = pd.DataFrame([{
        "WorkLevel": work,
        "Weather": weather,
        "SkinTone": skin,
        "Season": season
    }])

    probs = model.predict_proba(input_df)[0]
    classes = model.classes_

    top_idx = np.argmax(probs)
    top_fabric = classes[top_idx]
    print(f"\n✅ Recommended Fabric for {work}, {weather}, {skin}, {season}: {top_fabric}")

# --- Step 9: Example usage ---
recommend_fabric("Medium", "Hot", "Medium", "Summer")
recommend_fabric("High", "Cold", "Dark", "Winter")
recommend_fabric("Low", "Humid", "Fair", "Rainy")



 Accuracy: 96.00%
✅ Model saved as 'fabric_ml_pipeline.pkl'

✅ Recommended Fabric for Medium, Hot, Medium, Summer: Cotton

✅ Recommended Fabric for High, Cold, Dark, Winter: Wool

✅ Recommended Fabric for Low, Humid, Fair, Rainy: Satin
